In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
test_df = pd.read_csv('clean_test.csv')
test_df.fillna('NA',inplace=True)
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,SaleType,SaleCondition,AgeRemod
0,1461,20,RH,80.0,11622,Pave,NA,Reg,Lvl,AllPub,...,0,120,0,NA,MnPrv,NA,0,WD,Normal,0
1,1462,20,RL,81.0,14267,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,NA,Gar2,12500,WD,Normal,0
2,1463,60,RL,74.0,13830,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,MnPrv,NA,0,WD,Normal,1
3,1464,60,RL,78.0,9978,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,NA,NA,0,WD,Normal,0
4,1465,120,RL,43.0,5005,Pave,NA,IR1,HLS,AllPub,...,0,144,0,NA,NA,NA,0,WD,Normal,0


In [3]:
test_df.isna().any().sum()

0

In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1459 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [5]:
with open('cat_index.pkl','rb') as f:
    cat_index = pickle.load(f)

with open('vec_feat.pkl','rb') as f:
    vec_feat = pickle.load(f)
    
with open('columns.pkl','rb') as f:
    columns = pickle.load(f)

In [6]:
ids = test_df['Id'].values
test_df.drop(['Id'], axis=1, inplace=True)
X = test_df[columns].values
X.shape

(1459, 74)

In [7]:
norm_test_X = []
for i in range(X.shape[1]):
    if i in cat_index:
        result = vec_feat[i].transform(X[:,i].reshape(-1,1)).toarray()
#         print(i,result.shape)
    else:
        result = X[:,i].reshape(-1,1)
    
    if i > 0:
        norm_test_X = np.hstack((norm_test_X,result))
    else:
        norm_test_X = result

In [8]:
norm_test_X.shape

(1459, 281)

In [10]:
with open('column_fe_dict.pkl','rb') as f:
    column_dict = pickle.load(f)

In [9]:
with open('rf_model_fe.pkl','rb') as f:
    rf_model = pickle.load(f)

In [11]:
test_pred = rf_model.predict(norm_test_X[:,column_dict[67]])

In [12]:
test_pred_df = pd.DataFrame(columns=['SalePrice'], data=test_pred, index=ids)

In [13]:
test_pred_df.head()

,SalePrice
1461,137484.890
1462,160074.025
1463,168259.240
1464,176402.570
1465,203914.585


In [14]:
test_pred_df.to_csv('submission_fe_rf.csv',index_label='Id')